In [5]:
import numpy as np 
from citipy import citipy as cp 
from config import weather_api_key
import pandas as pd 
import requests
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add coordinates from tuple to list
coordinates = list(lat_lngs)

# Create a list for holding cities
cities = []

# Identify the nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = cp.nearest_city(coordinate[0], coordinate[1]).city_name
    # If city is not in list, add it to list
    if city not in cities:
        cities.append(city)
# Print count of cities to confirm sufficient count
len(cities)

757

In [6]:
# Create an empty list to hold the weather data
city_data = []
# Print th beginning of the logging
print("Beginning Data Retrieval  ")
print("--------------------------")

# Create counters
record_count = 1
set_count = 1

# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Loop through all the cities in the list using the enumerate method
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city
    city_url = url + '&q=' + city.replace(' ','+')

    # Log the URL, record, and set numbers and the city
    print(f'Processing Record {record_count} of Set {set_count} | {city}')
    # Increase the record count
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_desc = city_weather['weather'][0]['description']
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                            "Country": city_country,
                            "Lat": city_lat,
                            "Lng": city_lng,
                            "Max Temp": city_max_temp,
                            "Humidity": city_humidity,
                            "Cloudiness": city_clouds,
                            "Wind Speed": city_wind,
                            "Current Description": city_desc})

    # # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval  
--------------------------
Processing Record 1 of Set 1 | albany
Processing Record 2 of Set 1 | kruisfontein
Processing Record 3 of Set 1 | puerto ayora
Processing Record 4 of Set 1 | dikson
Processing Record 5 of Set 1 | los llanos de aridane
Processing Record 6 of Set 1 | mys shmidta
City not found. Skipping...
Processing Record 7 of Set 1 | qaanaaq
Processing Record 8 of Set 1 | sabha
Processing Record 9 of Set 1 | new norfolk
Processing Record 10 of Set 1 | cape town
Processing Record 11 of Set 1 | zhezkazgan
Processing Record 12 of Set 1 | vila franca do campo
Processing Record 13 of Set 1 | cherskiy
Processing Record 14 of Set 1 | verkhoyansk
Processing Record 15 of Set 1 | barrow
Processing Record 16 of Set 1 | ushuaia
Processing Record 17 of Set 1 | clyde river
Processing Record 18 of Set 1 | hilo
Processing Record 19 of Set 1 | tuktoyaktuk
Processing Record 20 of Set 1 | vila velha
Processing Record 21 of Set 1 | saint-philippe
Processing Record 22 o

In [7]:
# Convert city data array into DataFrame
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Albany,US,42.60,-73.97,57.00,87,86,1.23,overcast clouds
1,Kruisfontein,ZA,-34.00,24.73,63.97,64,98,6.08,overcast clouds
2,Puerto Ayora,EC,-0.74,-90.35,66.99,95,91,3.00,overcast clouds
3,Dikson,RU,73.51,80.55,20.03,96,61,10.54,broken clouds
4,Los Llanos De Aridane,ES,28.66,-17.92,68.00,60,0,8.05,clear sky
5,Qaanaaq,GL,77.48,-69.36,20.34,93,100,10.42,overcast clouds
6,Sabha,LY,27.04,14.43,63.10,45,0,7.74,clear sky
7,New Norfolk,AU,-42.78,147.06,66.99,60,100,3.58,overcast clouds
8,Cape Town,ZA,-33.93,18.42,61.00,93,40,4.70,scattered clouds
9,Zhezkazgan,KZ,47.80,67.71,33.44,75,0,9.64,clear sky


In [8]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")